<a href="https://colab.research.google.com/github/SoponUdomphon/my-learning-space/blob/main/cs-fundamental/01_memory_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛠️ **Hands-on Lab: Server Rescue Mission**
**สถานการณ์**: คุณพึ่งเข้าทำงานที่บริษัทแห่งหนึ่ง และได้รับมอบหมายให้แก้บั๊กในระบบจัดการนิยาย (Novel Management System) ที่มีปัญหาดังนี้:

1.  **Data Consistency:** แก้ข้อมูลนิยายเรื่องหนึ่ง แต่อีกเรื่องดันเปลี่ยนตาม?
2.  **Performance:** ระบบโหลดเนื้อหานิยายช้ามากจน User บ่น
3.  **Memory Leak:** Server แรมเต็มต้อง Restart ทุก 3 วัน


### **🛑 Station 1: The "Clone" Trap (Mutable & Reference)**
**โจทย์:** เรามี Cache เก็บรายชื่อนิยายแนะนำ แต่พอ Admin แก้ Tag ของ List A ทำไม List B (ที่เป็น Backup) ถึงเปลี่ยนตาม?

In [1]:
# 1. จำลองข้อมูล (Mutable Object)
top_novels = ["Harry Potter", "Percy Jackson"]
backup_novels = top_novels # <-- จุดเริ่มต้นของปัญหา

print(f"Original IDs: {id(top_novels)} vs {id(backup_novels)}")
print(f"Top & Backup IDs are same?: {top_novels is backup_novels}")

Original IDs: 139174532722048 vs 139174532722048
Top & Backup IDs are same?: True


In [2]:
# 2. เหตุการณ์: Admin เพิ่มนิยายเข้า List หลัก
top_novels.append("Lord of the Rings")

In [3]:
# 3. ตรวจสอบผลลัพธ์
print(f"Top: {top_novels}")
print(f"Backup: {backup_novels}") # ผลลัพธ์: Backup เปลี่ยนตามเฉยเลย!

Top: ['Harry Potter', 'Percy Jackson', 'Lord of the Rings']
Backup: ['Harry Potter', 'Percy Jackson', 'Lord of the Rings']


In [4]:
# ✅ Mission 1 Fix: จงแก้ไขบรรทัด 'backup_novels = ...'
# ให้เป็นการ Copy จริงๆ (ไม่ใช่แค่ส่ง Reference)
# Hint: ใช้ Slicing [:] หรือ .copy()

### ✅ **Station 1: Solution**

In [5]:
# 1. จำลองข้อมูล (Mutable Object)
top_novels = ["Harry Potter", "Percy Jackson"]
backup_novels = top_novels.copy() # <-- แก้ไขด้วยการใช้ .copy()

print(f"Original IDs: {id(top_novels)} vs {id(backup_novels)}")
print(f"Top & Backup IDs are same?: {top_novels is backup_novels}") # <- ตรวจสบ IDs ไม่ตรงกันแล้ว

# 2. เหตุการณ์: Admin เพิ่มนิยายเข้า List หลัก
top_novels.append("Lord of the Rings")

# 3. ตรวจสอบผลลัพธ์
print(f"Top: {top_novels}")
print(f"Backup: {backup_novels}") # ผลลัพธ์: Backup เปลี่ยนตามเฉยเลย!

Original IDs: 139174532746624 vs 139174532746432
Top & Backup IDs are same?: False
Top: ['Harry Potter', 'Percy Jackson', 'Lord of the Rings']
Backup: ['Harry Potter', 'Percy Jackson']


### **🐌 Station 2: The "Sloth" Loop (String Immutability)**
**โจทย์:** ฟังก์ชันรวมเนื้อหานิยายทำงานช้ามากกกก เพราะ Dev คนเก่าเขียนแบบ += ไว้

In [6]:
import time

# สร้างประโยคจำลอง 100,000 ประโยค
sentences = ["This is a sentence number " + str(i) for i in range(100000)]

def bad_concat(data):
    start = time.time()
    content = ""
    for s in data:
        content += s  # <--- คอขวด! สร้างบ้านใหม่ -> ย้ายของ -> ทุบบ้านเก่า (100,000 รอบ)
    print(f"Bad Way Time: {time.time() - start:.4f} seconds")

def good_concat(data):
    start = time.time()
    # ✅ Mission 2 Fix: เขียน Logic ใหม่โดยใช้ .join()
    content = ''.join(data) # <--- นับจำนวนของ -> สร้างบ้านตาม ขนาดจำนวนของ -> เอาของไปใส่ (1 รอบ)
    print(f"Good Way Time: {time.time() - start:.4f} seconds")

print("--- Testing Performance ---")
bad_concat(sentences)
good_concat(sentences) # ลองดูความต่างของเวลาครับ (ระดับ 10x - 100x)

--- Testing Performance ---
Bad Way Time: 0.0163 seconds
Good Way Time: 0.0033 seconds


### **👻 Station 3: The "Undead" Objects (Circular Ref & GC)**
**โจทย์:** ระบบจัดการ Chapter กินแรมไม่ยอมปล่อย เพราะแม่ (Novel) กับลูก (Chapter) กอดกันตาย (Circular Reference)

In [7]:
import weakref
import gc

class Novel:
    def __init__(self, title):
        self.title = title
        self.chapters = []
        print(f"Created Novel: {self.title}")

    def add_chapter(self, chapter):
        self.chapters.append(chapter)

    def __del__(self): # เมธอดที่จะทำงานเมื่อ Object ถูกลบจาก Memory (GC ทำงาน)
        print(f"💀 Destroyed Novel: {self.title}")

class Chapter:
    def __init__(self, title, parent):
        self.title = title
        # ❌ จุดเกิดเหตุ: Strong Reference กลับไปหาแม่
        self.parent = weakref.proxy(parent)
        # ✅ Mission 3 Fix: เปลี่ยนบรรทัดบนให้ใช้ weakref.proxy(parent)
        print(f"Created Chapter: {self.title}")

    def __del__(self):
        print(f"💀 Destroyed Chapter: {self.title}")

print("\n--- Testing Memory Leak ---")
def create_cycle():
    novel = Novel("Game of Thrones")
    chap1 = Chapter("Winter is Coming", novel) # ส่ง novel เข้าไปเป็น parent
    novel.add_chapter(chap1)

    # ตอนจบฟังก์ชัน ตัวแปร local (novel, chap1) จะถูกลบ
    # แต่ถ้ามันกอดกันเองอยู่ __del__ จะไม่ทำงาน (Memory Leak)
    return

create_cycle()
# สั่งเก็บขยะทันทีเพื่อเช็คผล (ถ้า Code ยังไม่แก้ จะไม่เห็นข้อความ Destroyed)
# gc.collect() # <-- สั่งแบบ manuale แต่ในกรณีนี้เราใช้ weakref.proxy แล้วไม่ต้องใช้ gc


--- Testing Memory Leak ---
Created Novel: Game of Thrones
Created Chapter: Winter is Coming
💀 Destroyed Novel: Game of Thrones
💀 Destroyed Chapter: Winter is Coming
